In [197]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [379]:
mydata=pd.read_csv('C:\Algo360\My python codes\Training files\BANK\messagetype\Debit\\bank_sms_classified_debit.csv',encoding='latin-1',sep='\t')
mydata.head()

CParserError: Error tokenizing data. C error: Expected 1 fields in line 655514, saw 2


In [352]:
mydata.count()
#print(mydata['domain_name'].value_counts().count())

message        301653
messagetype    301653
dtype: int64

In [353]:
mydata['messagetype'].value_counts()

Balance    301653
Name: messagetype, dtype: int64

In [354]:
#Preprocessing of train data
#mydata['length']=[len(str(x)) for x in mydata['message']]
mydata['messageprocessed'] = mydata['message'].str.replace(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b',
                                 'emailaddr')
mydata['messageprocessed'] = mydata['messageprocessed'].str.replace(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)',
                                  'httpaddr')
#mydata['messageprocessed'] = mydata['messageprocessed'].str.replace(r'£|\$|RS', 'moneysymb')    
mydata['messageprocessed'] = mydata['messageprocessed'].str.replace(
    r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b',
    'phonenumbr')    
mydata['messageprocessed'] = mydata['messageprocessed'].str.replace(r'\d+(\.\d+)?', 'numbr')
mydata['messageprocessed'] = mydata['messageprocessed'].str.replace(r'[^\w\d\s]', ' ')
mydata['messageprocessed'] = mydata['messageprocessed'].str.replace(r'\s+', ' ')
mydata['messageprocessed'] = mydata['messageprocessed'].str.replace(r'^\s+|\s+?$', '')
mydata['messageprocessed'] = mydata['messageprocessed'].str.lower()
#mydata['first'] = mydata.messageprocessed.str.split().str[0]
#mydata['first2']= mydata['messageprocessed'].apply(lambda x: " ".join(x for x in x.split()[:2]))
#mydata['last'] = mydata.messageprocessed.str.split().str[-1]
#mydata['last2']= mydata['messageprocessed'].apply(lambda x: " ".join(x for x in x.split()[-2:]))
mydata['wordcount'] =mydata['messageprocessed'].apply(lambda x: len(str(x).split(" ")))
#mydata['wordcount1'] = mydata['messageprocessed'].str.split().str.len()
mydata = mydata.sort_values(['messageprocessed'], ascending = [True])

In [355]:
mydata.count()

message             301653
messagetype         301653
messageprocessed    301653
wordcount           301653
dtype: int64

In [356]:
mydata = mydata.drop_duplicates('messageprocessed')
mydata.count()

message             26282
messagetype         26282
messageprocessed    26282
wordcount           26282
dtype: int64

In [15]:
#df_groupby=mydata.groupby(['last','first','wordcount']).head(1).reset_index(drop=True).sort_values(['first','last','wordcount'], ascending=False)

In [357]:
message_list = mydata["messageprocessed"].tolist()
prevmessage_list = mydata["messageprocessed"][:-1].tolist()
prevmessage_list=['']+prevmessage_list

In [358]:
len(message_list),len(prevmessage_list)

(26282, 26282)

In [359]:
se2 = pd.Series(prevmessage_list)
mydata['prevmessage']=se2.values

In [360]:
message_list[0],prevmessage_list[0]

(u'a c bal is rs numbr numbr cr and avl bal is rs numbr numbr', '')

In [361]:
fuzzratio=[]
for i in range(len(message_list)):
    fuzzratio.append(fuzz.ratio(message_list[i],prevmessage_list[i]))
se = pd.Series(fuzzratio)
mydata['fuzzratio']=se.values

In [362]:
import difflib

seqmatcherratio=[]
for i in range(len(message_list)):
    seq=difflib.SequenceMatcher(None,message_list[i],prevmessage_list[i])
    seqmatcherratio.append(seq.ratio()*100)
se1 = pd.Series(seqmatcherratio)
mydata['seqmatcherratio']=se1.values

In [363]:
#Function to perform Word by word match for each message with the previous message.
def wordbywordmatcher(mes1, mes2):
	#print 'Mess 1 : ',mes1
	#print 'Mess 2 : ',mes2
	mes1 = mes1.split()
	mes2 = mes2.split()
	len1 = len(mes1)
	curr1 = 0
	len2 = len(mes2)
	curr2 = 0
	match_count = 0
	new_match_msg = ""
	while(curr1 < len1 and curr2 < len2):
		curr_word1 = mes1[curr1].strip()
		curr_word2 = mes2[curr2].strip()
		#print curr_word1
		#print curr_word2
		#when there is exact match
		if curr_word1 == curr_word2:
			#print ' Direct Match Found'
			match_count+=1
			new_match_msg+=str(" "+curr_word1.strip())
		
		#Handling Cases when there is no exact match
		else:
			#Case when mes1: D A B C and mes2: D H A B C
			if (curr2+1 < len2) and (curr_word1 == mes2[curr2+1]):
				curr2+=1
				match_count+=1
				new_match_msg+=str(" "+curr_word1.strip())
			
			#Case when mes1: D A B C and mes2: D H T A B C
			elif (curr2+2 < len2) and (curr_word1 == mes2[curr2+2]):
				curr2+=2
				match_count+=1
				new_match_msg+=str(" "+curr_word1.strip())
				
			#Case when mes1: D H A B C and mes2: D A B C
			elif (curr1+1 < len1) and (curr_word2 == mes1[curr1+1]):
				curr1+=1
				match_count+=1
				new_match_msg+=str(" "+curr_word2.strip())
			
			#Case when mes1: D H T A B C and mes2: D A B C
			elif (curr1+2 < len1) and (curr_word2 == mes1[curr1+2]):
				curr1+=2
				match_count+=1
				new_match_msg+=str(" "+curr_word2.strip())
			
		curr1 += 1
		curr2 += 1
		pass
	#print match_count, new_match_msg
	return match_count, new_match_msg

#-------------------------------------------------------------------------------------------------------------------------------------------------

In [364]:
df_match = mydata[:]

leng = len(df_match)
msg_list = ['']
df_match.index = range(len(df_match))
wc_list = [0]

df_match['Message_Transformed'] = ''
df_match['Word_Match'] = 0
df_match.index = range(len(df_match))


for idx in range(1,leng,1):
	if idx % 1000 == 0:
		print idx
	last_message = df_match.at[idx-1,'messageprocessed']
	curr_message = df_match.at[idx,'messageprocessed']
	
	word_match, message_new = wordbywordmatcher(last_message, curr_message)
	msg_list.append(message_new)
	wc_list.append(word_match)
	
df_match['Message_Transformed'] = msg_list
df_match['Word_Match'] = wc_list
df_match['Perc_Match_with_last_message'] = (df_match['Word_Match']*100)/df_match['wordcount']
	

In [365]:
df_match[['messageprocessed','prevmessage','fuzzratio','seqmatcherratio','Perc_Match_with_last_message']].head()

,messageprocessed,prevmessage,fuzzratio,seqmatcherratio,Perc_Match_with_last_message
0,a c bal is rs numbr numbr cr and avl bal is rs...,,0,0.000000,0.000000
1,a c ca numbr opng numbr numbr numbr ledbal num...,a c bal is rs numbr numbr cr and avl bal is rs...,57,51.948052,35.294118
2,a c ca numbr opng numbr numbr numbr ledbal num...,a c ca numbr opng numbr numbr numbr ledbal num...,91,91.099476,94.117647
3,a c ca numbr opng numbr numbr numbr ledbal num...,a c ca numbr opng numbr numbr numbr ledbal num...,87,86.699507,84.210526
4,a c ca numbr opng numbr numbr numbr ledbal num...,a c ca numbr opng numbr numbr numbr ledbal num...,92,92.093023,94.736842


In [367]:
df_match.to_csv('C:\Algo360\My python codes\Training files\BANK\messagetype\Balance\Balancerawdump_Scores.csv', index=False)

In [368]:
df_match.count()

message                         26282
messagetype                     26282
messageprocessed                26282
wordcount                       26282
prevmessage                     26282
fuzzratio                       26282
seqmatcherratio                 26282
Message_Transformed             26282
Word_Match                      26282
Perc_Match_with_last_message    26282
dtype: int64

In [371]:
df_filtered=df_match[df_match['Perc_Match_with_last_message']<70]


In [373]:
df_filtered.to_csv('C:\Algo360\My python codes\Training files\BANK\messagetype\Balance\BalanceFiltered.csv', index=False)

In [369]:
df_match[df_match['fuzzratio']<70].count(),df_match[df_match['Perc_Match_with_last_message']<70].count()

(message                         1148
 messagetype                     1148
 messageprocessed                1148
 wordcount                       1148
 prevmessage                     1148
 fuzzratio                       1148
 seqmatcherratio                 1148
 Message_Transformed             1148
 Word_Match                      1148
 Perc_Match_with_last_message    1148
 dtype: int64, message                         2085
 messagetype                     2085
 messageprocessed                2085
 wordcount                       2085
 prevmessage                     2085
 fuzzratio                       2085
 seqmatcherratio                 2085
 Message_Transformed             2085
 Word_Match                      2085
 Perc_Match_with_last_message    2085
 dtype: int64)

In [345]:
df_match[df_match['Perc_Match_with_last_message']>-1].count()

message                         167916
messagetype                     167916
messageprocessed                167916
wordcount                       167916
prevmessage                     167916
fuzzratio                       167916
seqmatcherratio                 167916
Message_Transformed             167916
Word_Match                      167916
Perc_Match_with_last_message    167916
dtype: int64

In [370]:
df_match[df_match['seqmatcherratio']<70].count()

message                         3277
messagetype                     3277
messageprocessed                3277
wordcount                       3277
prevmessage                     3277
fuzzratio                       3277
seqmatcherratio                 3277
Message_Transformed             3277
Word_Match                      3277
Perc_Match_with_last_message    3277
dtype: int64